In [1]:
from D090_Ala_scan import *
#%run D090_Ala_scan.py
init()

/home/colin/code/pyrosetta_tests/D090_Ala_scan.py:80: UserWarning: Import of 'rosetta' as a top-level module is deprecated and may be removed in 2018, import via 'pyrosetta.rosetta'.
  from rosetta.protocols.scoring import Interface


PyRosetta-4 2020 [Rosetta PyRosetta4.conda.linux.CentOS.python37.Release 2020.10+release.46415fa3e9decb8b6e91a4e065c15543eb27a461 2020-03-05T09:09:24] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: {0} Checking for fconfig files in pwd and ./rosetta/flags
core.init: {0} Rosetta version: PyRosetta4.conda.linux.CentOS.python37.Release r248 2020.10+release.46415fa 46415fa3e9decb8b6e91a4e065c15543eb27a461 http://www.pyrosetta.org 2020-03-05T09:09:24
core.init: {0} command: PyRosetta -ex1 -ex2aro -database /home/colin/anaconda3/envs/pyrosetta/lib/python3.7/site-packages/pyrosetta/database
basic.random.init_random_generator: {0} 'RNG device' seed mode, using '/dev/urandom', seed=-2046964640 seed_offset=0 real_seed=-2046964640 thread_index=0
basic.random.init_random_generator: {0} RandomGenerator:init: Normal mode, seed=-2046964640 RG_type=mt19937


In [2]:
# performs general scanning editing here to make parallel
def scanning_parallel(pose_copy, partners, mutant_aa = 'A',
        interface_cutoff = 8.0, output = False,
        trials = 1, trial_output = ''):
    """
    Performs "scanning" at an interface within  <pdb_filename>  between
        <partners>  by mutating relevant residues to  <mutant_aa>  and repacking
        residues within  <pack_radius>  Angstroms, further repacking all
        residues within  <interface_cutoff>  of the interface residue, scoring
        the complex and subtracting the score of a pose with the partners
        separated by 500 Angstroms.
        <trials>  scans are performed (to average results) with summaries written
        to  <trial_output>_(trial#).txt.
        Structures are exported to a PyMOL instance.

    """
    # 1. set the pose
    pose = pose_copy

    # 2. setup the docking FoldTree and other related parameters
    dock_jump = 1
    movable_jumps = Vector1([dock_jump])
#    protocols.docking.setup_foldtree(pose, partners, movable_jumps)
# Commented out by colin because of error, found that API lists this method as below in PyRosetta4
    pyrosetta.rosetta.protocols.docking.setup_foldtree(pose, partners, movable_jumps)

    # 3. create ScoreFuncions for the Interface and "ddG" calculations
    # the pose's Energies objects MUST be updated for the Interface object to
    #    work normally
    scorefxn = get_fa_scorefxn() #  create_score_function('standard')
    scorefxn(pose)    # needed for proper Interface calculation

    # setup a "ddG" ScoreFunction, custom weights
    ddG_scorefxn = ScoreFunction()
    ddG_scorefxn.set_weight(core.scoring.fa_atr, 0.44)
    ddG_scorefxn.set_weight(core.scoring.fa_rep, 0.07)
    ddG_scorefxn.set_weight(core.scoring.fa_sol, 1.0)
    ddG_scorefxn.set_weight(core.scoring.hbond_bb_sc, 0.5)
    ddG_scorefxn.set_weight(core.scoring.hbond_sc, 1.0)

    # 4. create an Interface object for the pose
    interface = Interface(dock_jump)
    interface.distance(interface_cutoff)
    interface.calculate(pose)

    # 5. create a PyMOLMover for sending output to PyMOL (optional)
#     pymover = PyMOLMover()
#     pymover.keep_history(True)    # for multiple trajectories
#     pymover.apply(pose)
#     pymover.send_energy(pose)

    # 6. perform scanning trials
    # the large number of packing operations introduces a lot of variability,
    #    for best results, perform several trials and average the results,
    #    these score changes are useful to QUALITATIVELY defining "hotspot"
    #    residues
    # this script does not use a PyJobDistributor since no PDB files are output
    
    ddG_trials = []
    
    for trial in range( trials ):
        # store the ddG values in a dictionary
        ddG_mutants = {}
        for i in range(1, pose.total_residue() + 1):
            # for residues at the interface
            if interface.is_interface(i) == True:
                # this way you can TURN OFF output by providing False arguments
                #    (such as '', the default)
                filename = ''
                if output:
                    filename = pose.pdb_info().name()[:-4] + '_' +\
                        pose.sequence()[i-1] +\
                        str(pose.pdb_info().number(i)) + '->' + mutant_aa
                # determine the interace score change upon mutation
                key = pose.sequence()[i - 1] + str(pose.pdb_info().number(i)) + mutant_aa
                ddG_mutants[key] = interface_ddG(pose, i, mutant_aa,
                    movable_jumps, ddG_scorefxn, interface_cutoff, filename )
                
        ddG_trials.append(ddG_mutants)
        
    return ddG_trials

#         # output results
#         print( '='*80 )
#         print( 'Trial', str( trial + 1 ) )
#         print( 'Mutants (PDB numbered)\t\"ddG\" (interaction dependent score change)' )
#         residues = list( ddG_mutants.keys() )  # list(...) conversion is for python3 compatbility
#         residues.sort()    # easier to read
#         display = [pose.sequence()[i - 1] +
#             str(pose.pdb_info().number(i)) + mutant_aa + '\t' +
#             str(ddG_mutants[i]) + '\n'
#             for i in residues]
#         print( ''.join(display)[:-1] )
#         print( '='*80 )

#         # write to file
#         f = open(trial_output + '_' + str(trial + 1) + '.txt' , 'w' )
#         f.writelines(display)
#         f.close()

    #### alternate output using scanning_analysis (see below), only display
    ####    mutations with "deviant" score changes
    # commented out by colin
#     print( 'Likely Hotspot Residues' )
#     for hotspot in scanning_analysis(trial_output):
#         print( hotspot )
#     print( '='*80 )

# """
# 1.  creates a copy of the pose
# 2.  sets up a specific "ddG" ScoreFunction (if no ScoreFunction is provided)
# 3.  creates a copy of the pose to mutate
# 4.  mutates a single residue using mutate_residue
# 5.  calculates the "interaction energy" ( or "binding energy")
# 6.  outputs structures (optionally):
#         -to PyMOL
#         -to a PDB file
# """

In [ ]:
#test the method
pdb_filename = "./native_test.pdb"
original_pose = Pose()
pose_from_file(original_pose, pdb_filename)

Looks like Dask requires some pyrosetta.distributed packages. These appear to work best when things are submitted as rosettascripts. Will have to find some better sample rosettascripts later.

Try the pure python way:
``` python
from multiprocessing import Pool
import itertools
with pyrosetta.distributed.utility.log.LoggingContext(logging.getLogger("rosetta"), level=logging.WARN):
    with Pool() as p:
        work = [
            (input_pose, i, aa, "mutation")
            for i, aa in itertools.product(range(1, len(packed_pose.to_pose(input_pose).residues) + 1), IUPACData.protein_letters)
        ]
        logging.info("mutating")
        mutations = p.starmap(mutate_residue, work)
```

In [ ]:
from multiprocessing import Pool

In [ ]:
%%time
all_aas = ["A", "V", "I", "L", "L", "F", "Y", "W", "S", "T", "N", "Q", "G", "R", "H", "K", "D", "E"]
#all_aas = ["A", "V", 'I']
with Pool() as p:
    work = [(original_pose, "A_B", res, 8.0, False, 20) for res in all_aas]
    result = p.starmap(scanning_parallel, work)

Works!! Next is to parse the output into some nice data frames.

In [ ]:
import numpy as np
import pandas as pd
import seaborn

In [ ]:
def parse_AA_result(result):
    wt_all = []
    mut_all = []
    position_all = []
    scores = []
    for mut in result:
        for trial in mut:
            wt = [key[0] for key in trial.keys()]
            mutation = [key[-1] for key in trial.keys()]
            position = [int(key[1:-1]) for key in trial.keys()]
            score = [val for val in trial.values()]
            #print("{} {} {}".format(wt,mutation,position))
            wt_all += wt
            mut_all += mutation
            position_all += position
            scores += score
    df = pd.DataFrame()
    df['WT'] = wt_all
    df['Mutation'] = mut_all
    df['Position'] = position_all
    df['Score'] = scores
    return df

In [ ]:
df = parse_AA_result(result)
df.head()
print(df.shape)

In [ ]:
import matplotlib
matplotlib.pyplot.figure(figsize=(15,5))
seaborn.violinplot(x='Position', y='Score', data=df)

In [ ]:
df.to_csv(path_or_buf='all_AA_scan.csv')

In [ ]:
df.to_pickle(path='all_AA_scan.pkl')

In [ ]:
df_neg = df[df['Score']<5]
df_neg.head()

In [ ]:
matplotlib.pyplot.figure(figsize=(15,5))
seaborn.violinplot(x='Position', y='Score', data=df_neg)